In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [46]:
def load_binary_classif():
    from sklearn.datasets import load_breast_cancer
    from sklearn.preprocessing import StandardScaler
    X = load_breast_cancer().data
    y = load_breast_cancer().target
    st = StandardScaler()
    st.fit(X)
    X = st.transform(X)
    return X, y

def load_multiclassif():
    mnist = tf.keras.datasets.mnist
    (xtrain, ytrain), (xtest, ytest) = mnist.load_data()
    xtrain = xtrain / 255.
    xtest = xtest / 255.
    return xtrain, ytrain, xtest, ytest

In [22]:
class ANN():
    
    def __init__(self, layers, activation_function, kind, num_classes = None):
        """
        layers = list with the number of neurons per layers / the length of the list is the amount of hidden layers
        activation_function = from keras;
        kind = ['regression', 'binary_classif','multiclass_classif']
        """
        self.layers = layers
        self.activation_function = activation_function
        self.kind = kind
        self.num_classes = num_classes
        self.model = None
        self.res = None
    
    def fit(self, X, y, epochs, val_split, optimizer, metrics_tracking):
        D = X.shape[1]
        input_layer = tf.keras.layers.Input(shape = (D,))
        hidden_layers = [tf.keras.layers.Dense(x, activation = self.activation_function) for x in self.layers]
        
        if self.kind == 'regression':
            output_layer = tf.keras.layers.Dense(1)
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'mse', 
                          metrics = metrics_tracking)
            
        if self.kind == 'binary_classif':
            output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'binary_crossentropy', 
                          metrics = metrics_tracking)
            
        if self.kind == 'multiclass_classif':
            output_layer = tf.keras.layers.Dense(self.num_classes, activation = 'softmax')
            model = tf.keras.models.Sequential([input_layer] + hidden_layers + [output_layer])
            model.compile(optimizer = optimizer, 
                          loss = 'sparse_categorical_crossentropy', 
                          metrics = metrics_tracking)

        res = model.fit(X, y, epochs = epochs, validation_split = val_split)
        self.res = res
        self.model = model
        
    def predict(self, X):
        if self.model == None:
            print('Train it first.')
        else:
            return self.model.predict(X)

In [23]:
X, y = load_binary_classif()

In [24]:
ann = ANN(layers = [10,10,10], activation_function = 'relu', kind = 'binary_classif')

ann.fit(X, 
        y, 
        epochs = 10, 
        val_split = 0.33, 
        optimizer = tf.keras.optimizers.Adam(), 
        metrics_tracking = [tf.keras.metrics.binary_crossentropy])

Epoch 1/10
12/12 [==============================] - 1s 14ms/step - loss: 0.6792 - binary_crossentropy: 0.6792 - val_loss: 0.6096 - val_binary_crossentropy: 0.6096
Epoch 2/10
12/12 [==============================] - 0s 4ms/step - loss: 0.6493 - binary_crossentropy: 0.6493 - val_loss: 0.5687 - val_binary_crossentropy: 0.5687
Epoch 3/10
12/12 [==============================] - 0s 4ms/step - loss: 0.6132 - binary_crossentropy: 0.6132 - val_loss: 0.5222 - val_binary_crossentropy: 0.5222
Epoch 4/10
12/12 [==============================] - 0s 4ms/step - loss: 0.5738 - binary_crossentropy: 0.5738 - val_loss: 0.4669 - val_binary_crossentropy: 0.4669
Epoch 5/10
12/12 [==============================] - 0s 4ms/step - loss: 0.5269 - binary_crossentropy: 0.5269 - val_loss: 0.4108 - val_binary_crossentropy: 0.4108
Epoch 6/10
12/12 [==============================] - 0s 4ms/step - loss: 0.4770 - binary_crossentropy: 0.4770 - val_loss: 0.3530 - val_binary_crossentropy: 0.3530
Epoch 7/10
12/12 [=========

In [25]:
ann.predict(X)

18/18 [==============================] - 0s 941us/step


array([[0.32291326],
       [0.28186044],
       [0.28959933],
       [0.19270492],
       [0.41823527],
       [0.42149192],
       [0.27958336],
       [0.508322  ],
       [0.32569408],
       [0.23712544],
       [0.31056648],
       [0.34016618],
       [0.2942936 ],
       [0.38624686],
       [0.3279023 ],
       [0.26670778],
       [0.35620016],
       [0.30689484],
       [0.1996358 ],
       [0.823601  ],
       [0.816471  ],
       [0.98589736],
       [0.40214613],
       [0.15357557],
       [0.2018371 ],
       [0.31668875],
       [0.31206602],
       [0.33759612],
       [0.2595451 ],
       [0.43592197],
       [0.22688505],
       [0.40174145],
       [0.27784306],
       [0.2324819 ],
       [0.34210783],
       [0.21391694],
       [0.37581244],
       [0.9880352 ],
       [0.9151979 ],
       [0.4227894 ],
       [0.8758904 ],
       [0.48729682],
       [0.1993592 ],
       [0.40743446],
       [0.4359713 ],
       [0.32731292],
       [0.98233896],
       [0.358

In [47]:
Xtrain, ytrain, Xtest, ytest = load_multiclassif()
Xtrain = Xtrain.reshape(Xtrain.shape[0], -1)
Xtest = Xtest.reshape(Xtest.shape[0], -1)

In [48]:
len(np.unique(y))

10

In [49]:
ann = ANN(layers = [10,10,10], activation_function = 'relu', kind = 'multiclass_classif', num_classes = 10)

ann.fit(Xtrain, 
        ytrain, 
        epochs = 10, 
        val_split = 0.33, 
        optimizer = tf.keras.optimizers.Adam(), 
        metrics_tracking = [tf.keras.metrics.sparse_categorical_crossentropy])

Epoch 1/10
1257/1257 [==============================] - 3s 2ms/step - loss: 0.7050 - sparse_categorical_crossentropy: 0.7050 - val_loss: 0.4073 - val_sparse_categorical_crossentropy: 0.4073
Epoch 2/10
1257/1257 [==============================] - 3s 2ms/step - loss: 0.3550 - sparse_categorical_crossentropy: 0.3550 - val_loss: 0.3233 - val_sparse_categorical_crossentropy: 0.3233
Epoch 3/10
1257/1257 [==============================] - 3s 2ms/step - loss: 0.3070 - sparse_categorical_crossentropy: 0.3070 - val_loss: 0.3044 - val_sparse_categorical_crossentropy: 0.3044
Epoch 4/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.2766 - sparse_categorical_crossentropy: 0.2766 - val_loss: 0.2863 - val_sparse_categorical_crossentropy: 0.2863
Epoch 5/10
1257/1257 [==============================] - 2s 2ms/step - loss: 0.2533 - sparse_categorical_crossentropy: 0.2533 - val_loss: 0.3007 - val_sparse_categorical_crossentropy: 0.3007
Epoch 6/10
1257/1257 [============================

In [51]:
ann.predict(Xtest).shape

313/313 [==============================] - 0s 891us/step


(10000, 10)

In [52]:
ann.predict(Xtest)

313/313 [==============================] - 0s 888us/step


numpy.ndarray